# Metagenomics Bioinformatics Course - EBI MGnify 2021
## MGnify Genomes resource - Metagenomic Assembled Genomes Catalogues - Practical exercise

### Aims
In this exercise, we will learn how to use the [Genomes resource within MGnify](https://www.ebi.ac.uk/metagenomics/browse#genomes).

- Discover the available data on the MGnify website
- Use two search mechanisms (search by _gene_ and search by _genome_)
- Learn how to use the MGnify API to fetch data using scripts or analysis notebooks
- Use the _genome_ search mechanism via the API, to compare your own MAGs against a MGnify catalogue and see whether they are novel

### How this works
This file is a [Jupyter Notebook](https://jupyter.org). 
It has instructions, and also code cells. The code cells are connected to Python, and you can run all of the code in a cell by pressing Play (▶) icon in the top bar, or pressing `shift + return`.
The code libraries you should need are already installed.

# Import packages

[pandas](https://pandas.pydata.org/docs/reference/index.html#api) is a data analysis library with a huge list of features. It is very good at holding and manipulating table data. It is almost always short-handed to `pd`

In [ ]:
import pandas as pd

[jsonapi-client](https://pypi.org/project/jsonapi-client/) is a library to get formatted data from web services into python code

In [ ]:
from jsonapi_client import Session as APISession

[matplotlib](https://matplotlib.org) is the go-to package for making plots and charts. It is almost always short-handed to `plt`.

In [ ]:
import matplotlib.pyplot as plt

`pathlib` is part of the Python standard library. We use it to find files and directories.

In [ ]:
from pathlib import Path

`time` is part of the Python standard library. We will use it to wait for results from the API.

In [ ]:
import time

`tarfile` is part of the Python standard library. We will use it to extract compressed files from a `.tar.gz` file that the API gives us.

In [ ]:
import tarfile

**We will also import some extra package later. `sourmash` and `requests` will be used for specialised tasks and explained at the time.**

# The MGnify API (recap from [day 2](<./Day 2 - Service API Practical.ipynb>))
<span style="background-color:#ffaaaa; padding: 8px">Saw all this on Day 2? [⇓ Skip to new tasks](#Task-1---list-Genome-Catalogues).</span>

## Core concepts
An [API](https://en.wikipedia.org/wiki/API "Application programming interface") is how your scripts (e.g. Python or R) can talk to the MGnify database.

The MGnify API uses [JSON](https://en.wikipedia.org/wiki/JSON "Javascript Object Notation") to transfer data in a systematic way. This is human-readable and computer-readable.

The particular format we use is a standard called [JSON:API](https://jsonapi.org). 
There is a Python package ([`jsonapi_client`](https://pypi.org/project/jsonapi-client/)) to make consuming this data easy. We're using it here.

The MGnify API has a "browsable interface", which is a human-friendly way of exploring the API. The URLs for the browsable API are exactly the same as you'd use in a script or code; but when you open those URLs in a browser you see a nice interface. Find it here: [https://www.ebi.ac.uk/metagenomics/api/v1/](https://www.ebi.ac.uk/metagenomics/api/v1/).

The MGnify API is "paginated", i.e. when you list some data you are given it in multiple pages. This is because there can sometimes by thousands of results. Thankfully `jsonapi_client` handles this for us.

## Example
The MGnify website has a list of ["Super Studies"](https://www.ebi.ac.uk/metagenomics/browse) (collections of studies that together represent major research efforts or collaborations).

What the website is actually showing, is the data from an API endpoint (i.e. specific resource within the API) that lists those. It's here: [api/v1/super-studies](https://www.ebi.ac.uk/metagenomics/api/v1/super-studies). Have a look.

Here is an example of some Python code, using two popular packages that let us write a short tidy piece of code:

![Do this step](assets/action.png) **Click into the next cell, and press `shift + return` (or click the ▶ icon on the menubar at the top) to run it.**

In [ ]:
endpoint = 'super-studies'

with APISession("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    resources = map(lambda r: r.json, mgnify.iterate(endpoint))
    resources = pd.json_normalize(resources)
    resources.to_csv(f"{endpoint}.csv")
resources

## Line by line explanation

```python
### The packages were already imported, but if you wanted to use this snippet on it's own as a script you would import them like this:
from jsonapi_client import Session as APISession
import pandas as pd
###


endpoint = 'super-studies'
# An "endpoint" is the specific resource within the API which we want to get data from. 
# It is the a URL relative to the "server base URL" of the API, which for MGnify is https://www.ebi.ac.uk/metagenomics/api/v1.
# You can find the endpoints in the API Docs https://www.ebi.ac.uk/metagenomics/api/docs/ 

with APISession("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    # Calling "APISession" is enabling a connection to the MGnify API, that can be used multiple times. 
    # The `with...as mgnify` syntax is a Python "context". 
    # Everything inside the `with...` block (i.e. indented below it) can use the `APISession` which we've called `mgnify` here. 
    # When the `with` block closes (the indentation stops), the connection to the API is nicely cleaned up for us.
    
    resources = map(lambda r: r.json, mgnify.iterate(endpoint))
    # `map` applies a function to every element of an iterable - so do something to each thing in a list.
    # Remember we said the API is paginated? 
    # `mgnify.iterate(endpoint)` is a very helpful function that loops over as many pages of results as there are.
    # `lambda r: r.json` is grabbing the JSON attribute from each Super Study returned from the API.
    # All together, this makes `resources` be a bunch of JSON representations we could loop through, each containing the data of a Super Study.
    
    resources = pd.json_normalize(resources)
    # `pd` is the de-facto shorthand for the `pandas` package - you'll see it anywhere people are using pandas.
    # The `json_normalize` function takes "nested" data and does its best to turn it into a table.
    # You can throw quite strange-looking data at it and it usually does something sensible.
    
    resources.to_csv(f"{endpoint}.csv")
    # Pandas has a built-in way of writing CSV (or TSV, etc) files, which is helpful for getting data into other tools.
    # This writes the table-ified Super Study list to a file called `super-studies.csv`.
    
resources
# In a Jupyter notebook, you can just write a variable name in a cell (or the last line of a long cell), and it will print it.
# Jupyter knows how to display Pandas tables (actually called "DataFrames", because they are More Than Just Tables ™) in a pretty way.
```


# Day 4 Tasks
## Task 1 - list Genome Catalogues
![Do this step](assets/action.png) **In the cell below, complete the Python code to fetch the list of [Genome Catalogues from the MGnify API](https://www.ebi.ac.uk/metagenomics/api/v1/genome-catalogues), and show them in a table.**

(Note that there may only be one catalogue in the list right now, that is correct)

In [ ]:
# In case we skipped the API recap, make sure packages are imported
import pandas as pd
from jsonapi_client import Session as APISession
import time
import tarfile
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# Complete this code

endpoint = 

with APISession("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    catalogues = 
    
    
    
    
catalogues

### Solution

Unhide these cells to see a solution

In [ ]:
endpoint = 'genome-catalogues'

with APISession("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    catalogues = map(lambda r: r.json, mgnify.iterate(endpoint))
    catalogues = pd.json_normalize(catalogues)
catalogues

## Task 2 - list Genomes
Each catalogue contains a much larger list of Genomes.

![Do this step](assets/action.png)  **In the cell below, complete the Python code to fetch the list of [Genomes from the MGnify API](https://www.ebi.ac.uk/metagenomics/api/v1/genome-catalogues), and show them in a table.**

(Note that there are quite a lot of pages of data, so this will take a minute to run)

In [ ]:
catalogue_id = catalogues.id[0]
endpoint = f'genome-catalogues/{catalogue_id}/genomes'  # a Python f-string inserts the value of a variable into the string where that variable name appears inside {..}

with           as mgnify:
    genomes = 

    
    
    
genomes

### Solution
Unhide these cells to see a solution

In [ ]:
catalogue_id = catalogues.id[0]
endpoint = f'genome-catalogues/{catalogue_id}/genomes'  # a Python f-string inserts the value of a variable into the string where that variable name appears inside {..}

with APISession("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    genomes = map(lambda r: r.json, mgnify.iterate(endpoint))
    genomes = pd.json_normalize(genomes)
genomes

## Task 3 - search Genome Catalogues using the website
[MGnify Genomes](https://www.ebi.ac.uk/metagenomics/browse#genomes) offers two ways to search for Genomes in a MAG Catalogues.

### Search using a gene
1. ![Do this step](assets/action.png) Go to [the MGnify Genomes webpage](https://www.ebi.ac.uk/metagenomics/browse#genomes) and open the latest version of the Human Gut catalogue.
2. Imagine you've got some sequence of interest and want to see whether it is in the Gut Genome Catalogue. 
    * ![Do this step](assets/action.png)  **Use the following sequence, and the "Search by gene" tool** to discover whether is it likely to from a species in the gut.

    ```
    GGAGTGCGGCGGAAAGTTAACCTATGCCGGACCCTGCGGGAATCCAGCTGCGTTCGAACAAGCAACCAACATATATATCTGAATTTGGATGTGGTGGGCACTTTGT
    TGTTAGGCGCTTTGAGGTGCGAGTGACACTTTGGGGTGCGCGGAGCCCTGGGTTGGGTCGATGATTTGGGATGAGCTTCTTACTTAGGTGAAGAGGGGCTTTATGG
    CTGAGAGGTAGTCTTTGGCTACGTCGGCTTTATCTGCTTGGAAATTGTGCCAGGCCCACCATTGGACCATTCCTACGAAGCTTGAGGCTATGTGGTGTAGTAGGAA
    GCTTCGGTCCATGGTGGCGGCGGGGCCGTTTGGGTCGGTAGGGACGGTTTCGGCTGCTCGGGCCATGATGGTCTTGCGGAGGCTGTCGGCGAAGACGCGTGAACCG
    GCGCCGGCTACCAGTGCCCGTACACCCTGACGGCGCTCCCAGAGGTTGTTGAGGATATGCTCGACCTGTACGAGTGGGTCATCGAGGGGCGTACCGTCATCGTCGA
    GGGCATGGGTGCAGATATCGCGCACGAGCTCAGCGAGCAGGTCATCTTTGCTTTTGAAGAGGCCGTAGAACGTGGCCCGACCCACATGGGCGCGAGCGATGATGTC
    GCCGACGGTGATCTTGCCGTAGTCCTCTTCGCGCAGCAGCTCGGAGAACGCCGCGACGATCGCGGCGCGGCTTTTGGCCTTGCGGGCATCCATGGCTATGCGTCCG
    CGTCAACGAGCAGACAGCGGAGCGTCCCGGAGCAGCCCTCGTAGGGGCGCTTGCCGGCGCCGTAGCCGACGGCTTCGATGCGGTAGCGTGAGGGCAGTTGGTCGGA
    CGTGCGCAGAGCAGTGACGATGGCGGCGCCCGTGGGCGTCACGAGCTCGCCGGCGACCGGTGCAGGCGTGAGGGCGATATTGCCCGCCTGGCACAGGTTGACGACA
    GCGGGGACGGGAATGGGCATGAGGCCGTGGGCGCAGCGAATGGCGCCGTGGCCCTCGAAAAGCGAC
    ```

    * ![Consider this](assets/question.png) This search compares [k-mers](https://en.wikipedia.org/wiki/K-mer) of length 31, from your search sequence to every genome in the catalogue. Look at the `% Kmers found` column in the results, and the BLAST score *p* values. **Do you think the top hit is a certain match?**
    * ![Consider this](assets/question.png) Click the Genome Accession of the top hit. Browse the available information for that Genome. What do you think the role of this species is in the human gut?

### Search using a genome
For the last couple of tasks, you will need your binned MAGs generated on Day 3 of the course.
If you didn't finish that practical, then you can copy some we made from the `penelopeprime` shared drive.

1. ![Do this step](assets/action.png)  Put your binned MAG fasta files in the folder `/home/training/mags`.
    * If you don't have them, run this command in a Terminal: `cp -r /media/penelopeprime/Metagenomics-Nov21/Day4/day4-example-mag-bins/* ~/mags`
2. ![Do this step](assets/action.png)  Go back to [the MGnify Genomes webpage](https://www.ebi.ac.uk/metagenomics/browse#genomes) and open the latest version of the Human Gut catalogue.
3. ![Do this step](assets/action.png)  Pick one of your binned MAGs, and use its FASTA file with the "Search by MAG" tool on the website to see whether your MAG is similar to any of those in the catalogue.
    * The query might take a couple of minutes to be queued and run.
    * Once it finishes, look at the results table. 
        * This search uses [sourmash](https://sourmash.readthedocs.io/en/latest/) to find how much of your query metagenome is contained by target genomes in the MAG catalogue.
        * A result where the best match shows "60% query covered" means 60% of the query MAG's partitions were found in the best matching catalogue MAG.
        * Download the CSV file of all the matches (there is an icon in the results table).
        * The [Sourmash documentation](https://sourmash.readthedocs.io/en/latest/classifying-signatures.html#appendix-a-how-sourmash-gather-works) explains the columns in this table.
4. ![Do this step](assets/action.png)  Calculate the total ammount (i.e. the `sum`) of your query MAG that is covered by MAGs from the catalogue, by fixing the second half of code snippet (adding a calculation).

In [ ]:
##### FIX ME #####
downloaded_csv_file = '/home/training/downloads/                                           .csv'
##################

sourmash_results = pd.read_csv(downloaded_csv_file)
display(sourmash_results)  # this shows the CSV table, loaded into a Pandas dataframe, in a pretty format. `display` is a special Jupyter function, that won't work in a regular python script.

query_contained_by_best_match = sourmash_results.f_unique_to_query.max()
print(f'The best matching MAG contained {query_contained_by_best_match * 100}% of the query’s k-mers.')


##### FIX ME #####
query_contained_by_all_matches = 
print(f'All matching MAGs together contained {query_contained_by_all_matches * 100}% of the query’s k-mers.')
##################


![Consider this](assets/question.png) Do you think this containment fraction means your MAG is novel, or is it well-represented by genomes in the MGnify catalogue? How complete and contaminated do you think your MAGs are? How would a low completeness (say 50%) affect the threshold you’d be looking for to consider your MAG represented by the catalogue?

### Solution
Unhide these cells to see a solution.

In [ ]:
import os
downloaded_csv_file = max(Path('/home/training/Downloads').glob('*.csv'), key=os.path.getctime)

sourmash_results = pd.read_csv(downloaded_csv_file)
display(sourmash_results)  # this shows the CSV table, loaded into a Pandas dataframe, in a pretty format. `display` is a special Jupyter function, that won't work in a regular python script.

query_contained_by_best_match = sourmash_results.f_unique_to_query.max()
print(f'The best matching MAG contained {query_contained_by_best_match * 100}% of the query’s k-mers.')

query_contained_by_all_matches = sourmash_results.f_unique_to_query.sum()
print(f'All matching MAGs together contained {query_contained_by_all_matches * 100}% of the query’s k-mers.')

## Task 4 - Find out whether your MAGs are novel, using the API
In this final task, we will combine the API skills you’ve gained with your knowledge of the MAG search mechanism.

Imagine you created more than a couple of MAGs, following the process of Day 3 but using a big dataset and a high performance computing cluster. Now, you want to see if any of them are novel or are they well covered by a catalogue on the MGnify resource. It will be a pain to do all that by hand! You can upload a directory of a few MAGs on the website, but for 100s or 1000s, you need to use the API.

Follow along and fill in the missing pieces of code to do this.

We need to compute a "sketch" for each Genome, using Sourmash. On the website this happens in your browser. To use the API, we do it using the [sourmash](https://sourmash.readthedocs.io/en/latest/) package. We will also use [Biopython’s SeqIO](https://biopython.org/wiki/SeqRecord) package to read the FASTA files in Python code. 

**Most of the code here is completed for you, because it would take some time to learn how to put it all together. You can follow the code comments, or just press `shift + enter` in each cell to run it and come back to study it later.**

### Load up our MAGs

In [ ]:
# If you didn't have these packages installed, you'd need to run "pip install pandas biopython sourmash"

import sourmash
from Bio import SeqIO

We’ll find the filepath for all of our "new" MAGs.

![Do this step](assets/action.png) Edit the value of `my_mags_folder` if you put your MAGs somewhere different.

In [ ]:
my_mags_folder = Path('/home/training/mags')

# pathlib is a handy standard Python library for finding files and directories
my_mags_files = list(my_mags_folder.glob('*.fa*'))
my_mags_files

### Calculate sourmash "sketches" to search against the MGnify catalogue
We’ll compute a sourmash sketch for each MAG. A sketch goes into a signature, that we will use for searching. The signature is a sort of collection of hashes that are well suited for calculating the *containment* of your MAGs within the catalogue's MAGs.

In [ ]:
for mag_file in my_mags_files:
    # the sourmash parameters are chosen to match those used within MGnify
    sketch = sourmash.MinHash(n=0, ksize=31, scaled=1000)
    
    # a fasta file may have multiple records in it. add them all to the sourmash signature.
    for index, record in enumerate(SeqIO.parse(mag_file, 'fasta')):
        sketch.add_sequence(str(record.seq))

    # save the sourmash sketch as a "signature" file
    sig = sourmash.SourmashSignature(sketch, name=record.name or mag_file.stem)
    with open(mag_file.stem + '.sig', 'wt') as fp:
        sourmash.save_signatures([sig], fp)

# check what signature files we've created.
# using ! in Jupyter lets you run a shell command. It is handy for quick things like pwd and ls.
!ls *.sig

### Submit a search job to the MGnify API
We’ll call the MGnify API with all of our sketches.
There is an endpoint for this (the same one used by the website).

In this case, we need to **send** data to the API (not just fetch it). This is called "POST"ing data in the API world. 

This part of the API is quite specialized and so is not a formal JSON:API, so we use the more flexible [requests](https://docs.python-requests.org/en/master/) Python package to communicate with it.

In [ ]:
import requests

In [ ]:
endpoint = 'https://www.ebi.ac.uk/metagenomics/api/v1/genomes-search/gather'
catalogue_id = 'human-gut-v1-0'  # You could change this to any other catalogue ID from the MGnify website, if you use this in the future.

# Create a list of file uploads, and attach them to the API request
signatures = [open(mag.stem + '.sig', 'rb') for mag in my_mags_files]
sketch_uploads = [('file_uploaded', signature) for signature in signatures]

# Send the API request - it specifies which catalogue to search against and attaches all of the signature files.
submitted_job = requests.post(endpoint, data={'mag_catalog': catalogue_id}, files=sketch_uploads).json()

map(lambda fp: fp.close(), signatures)  # tidy up open file pointers

print(submitted_job)

### Wait for our results to be ready
As you can see in the printed `submitted_job` above, a `status_URL` was returned in the response from submitting the job via the API.
Since the job will be in a queue, we must poll this `status_URL` to wait for our job to be completed.
We’ll check every 2 seconds until ALL of the jobs are finished.

In [ ]:
job_done = False
while not job_done:
    print('Checking status...')
    status = requests.get(submitted_job['data']['status_URL'])
    # the status_URL is just another API endpoint that's unique for our search job
    
    queries_done = {sig['job_id']: sig['status'] for sig in status.json()['data']['signatures']}
    job_done = all(map(lambda q: q == 'SUCCESS', queries_done.values()))
    if not job_done:
        print('Still waiting for jobs to complete. Current status of jobs')
        print(queries_done)
        print('Will check again in 2 seconds')
        time.sleep(2)

print('All finished!')

### Download all of the search results
Now, we need to fetch the results. We can grab these all at once, as a compressed archive (a `.tgz` file), via the top level `results_url` from the `status` endpoint's response.

In [ ]:
results_endpoint = status.json()['data']['results_url']
print(f'Will fetch results from {results_endpoint}')

results_response = requests.get(results_endpoint, stream=True)
assert results_response.status_code == 200

with open('mag_novelty_results.tgz', 'wb') as tarball:
    tarball.write(results_response.raw.read())

!ls *.tgz

### Make a table with our search results
The tarball we just downloaded contains `.csv` files – one for each query, so one for each of your MAGs.
We can load them all up and put them into a single Pandas dataframe:

In [ ]:
# we'll make an array of all the tables and concatenate them using pandas
results_tables = []

# We need to translate the Job IDs (assigned by the MGnify API) back to the name of the each MAG.
# This just creates that map, so in the combined table we know what result applies to what MAG.
job_to_mag_filename = {sig['job_id']: sig['filename'].rstrip('.sig') for sig in status.json()['data']['signatures']}

# Python has built-in support for tarfiles, so we can pull the CSVs from it straight into Pandas.
with tarfile.open('mag_novelty_results.tgz', 'r:gz') as tarball:
    for results_csv in tarball.getmembers():
        results_table = pd.read_csv(tarball.extractfile(results_csv))
        
        # add a column to the table with the MAG filename on every row
        job_id = results_csv.name.rstrip('.csv')
        results_table['query_mag'] = job_to_mag_filename[job_id]
        results_tables.append(results_table)

# Stick all the tables together (same columns, so we're stacking the rows here)
mag_novelty_results = pd.concat(results_tables, ignore_index=True)

mag_novelty_results

### Compute statistics on the search results
We can find the number of matches for each MAG by grouping and counting the table rows.

We will use Pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) for this. GroupBy lets you calculate an "aggregate" statistic on each group of rows, where the groups are usually defined by having the same value of some column. In our case, we're grouping by the `query_mag` column.

In [ ]:
mag_novelty_results.groupby('query_mag').count()

![Do this step](assets/action.png) **Calculate the apparent novelty of each of your MAGs**. 
Use a `groupby`, and remember you’re trying to find out how much of each MAG is contained by ALL of the matches from the catalogue.
That is the `f_unique_to_query` column.

In [ ]:
mag_novelty_results.groupby(

#### Solution
Unhide these cells to see a solution.

In [ ]:
mag_novelty_results.groupby('query_mag').sum().sort_values('f_unique_to_query')
# We've added the .sort_values() for bonus points! This just orders the table by that column.

### Examine properties of matched MAGs, using data from the MGnify FTP
Finally, let’s look at the matched catalogue genomes for one of our MAGs.

Since we're looking at *containment* of your MAGs within a catalogue, the sourmash search looks at **all** genomes in the MGnify catalogue, not just the cluster/species representatives that are browsable on the website and accessible by the API.

The [EBI FTP site for MGnify](http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/) lets you access the full catalogue dataset.
There is a file for each catalogue (`genomes-all_metadata.tsv`) with some basic information about all of the genomes. This file also contains the mapping of each genome to its Species Representative (as shown on the website and API).

We’ve downloaded it to the shared drive (`penelopeprime`) to save you a few minutes waiting for it.

For this task we will:
- find which of your MAGs has the most matches from the sourmash search
- extract the ID of those matches, from the search results
- find the corresponding rows in the big metadata table we fetched from the FTP
- plot statistics about the matched MGnify genomes

![Do this step](assets/action.png) **Explore, and complete, the code in the following cells to do this**

In [ ]:
all_genomes_metadata = pd.read_csv('/media/penelopeprime/Metagenomics-Nov21/Day4/genomes-all_metadata.tsv', sep='\t', index_col='Genome')
# the (big) file was downloaded from http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v1.0/genomes-all_metadata.tsv
all_genomes_metadata

In [ ]:
# Find the mag with most matches (use a .groupby() and an aggregate statistic that counts the rows of each group)
#   .f_unique_to_query.idxmax() finds the INDEX (label) corresponding to the maximum vaule of the f_unique_to_query column. 
#   Note that any column would do since we're just counting!
#   We want mag_with_most_matches to be something like "bin.2".

########################################### COMPLETE ME ###########################################
mag_with_most_matches = mag_novelty_results.                            .f_unique_to_query.idxmax()
###################################################################################################

print(f'{mag_with_most_matches} has the most matches')

In [ ]:
# Pull out the matched genome name from the filepath that's returned in the results
mag_novelty_results['match_genome_name'] = mag_novelty_results.apply(lambda result: result['name'].split('/')[-1].rstrip('.fa'), axis=1)

In [ ]:
# Select the search results for the MAG we're interested in – the one with the most matches
matches_of_interest = mag_novelty_results[mag_novelty_results.query_mag==mag_with_most_matches]

# Use the newly created "match_genome_name" as the table index
# we can do this because it is now unique (the same MGnify genome can't match twice for the same query)
matches_of_interest.set_index('match_genome_name', inplace=True)
matches_of_interest

In [ ]:
# Pandas has a powerful "join" feature. Since we now have two tables of data indexed by the MGnify genome IDs, we can join their columns together
matches_of_interest = matches_of_interest.join(all_genomes_metadata)

matches_of_interest

In [ ]:
# Let’s see the completeness of the MGnify genomes that seem to contain our most-matched MAG
plt.hist(matches_of_interest.Completeness)
plt.xlabel('Completeness \ %')
plt.ylabel('Number of genomes')
plt.title(f'Completeness histogram for MGnify Genomes matching {mag_with_most_matches}');

In [ ]:
# Make a plot of the contamination fraction of each of the genomes that match our MAG.
# Note that you can call `matches_of_interest.columns` to see a list of all the columns in the table.

########################################### COMPLETE ME ###########################################
plt.
plt.xlabel('Contamination \ %')
plt.ylabel
plt.title
###################################################################################################

In [ ]:
# Make a chart showing the which continents had how many samples matching your MAG
########################################### COMPLETE ME ###########################################





###################################################################################################

#### Solution
Unhide these cells to see a solution

In [ ]:
import matplotlib.pyplot as plt

all_genomes_metadata = pd.read_csv('/Users/sandyr/Downloads/genomes-all_metadata.tsv', sep='\t', index_col='Genome')
# the (big) file was downloaded from http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v1.0/genomes-all_metadata.tsv

mag_with_most_matches = mag_novelty_results.groupby('query_mag').count().f_unique_to_query.idxmax()
print(f'{mag_with_most_matches} has the most matches')

# Pull out the matched genome name from the filepath that's returned in the results
mag_novelty_results['match_genome_name'] = mag_novelty_results.apply(lambda result: result['name'].split('/')[-1].rstrip('.fa'), axis=1)

# Select the search results for the MAG we're interested in – the one with the most matches
matches_of_interest = mag_novelty_results[mag_novelty_results.query_mag==mag_with_most_matches]

# Use the newly created "match_genome_name" as the table index
# we can do this because it is now unique (the same MGnify genome can't match twice for the same query)
matches_of_interest.set_index('match_genome_name', inplace=True)

# Pandas has a powerful "join" feature. Since we now have two tables of data indexed by the MGnify genome IDs, we can join their columns together
matches_of_interest = matches_of_interest.join(all_genomes_metadata)

# Let’s see the completeness of the MGnify genomes that seem to contain our most-matched MAG
plt.figure(0)
plt.hist(matches_of_interest.Completeness)
plt.xlabel('Completeness \ %')
plt.ylabel('Number of genomes')
plt.title(f'Completeness histogram for MGnify Genomes matching {mag_with_most_matches}');

plt.figure(1)
plt.hist(matches_of_interest.Contamination)
plt.xlabel('Contamination \ %')
plt.ylabel('Number of genomes')
plt.title(f'Contamination histogram for MGnify Genomes matching {mag_with_most_matches}');

plt.figure(2)
matches_of_interest.Continent.hist()
plt.xlabel('Continent where sample was collected')
plt.ylabel('Number of samples')
plt.title(f'Geographical spread of samples for MGnify Genomes matching {mag_with_most_matches}');